In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.callbacks import TensorBoard
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.models import model_from_json


input_encoder = Input(shape=(28, 28, 1))
input_decoder = Input(shape=(4, 4, 4))


def custom_activation(x):
    return (K.sigmoid(x) * 3) 

with open('decoder.json', 'r') as f:
    decoder = model_from_json(f.read())
    decoder.load_weights("decoder.h5")
    
with open('encoder.json', 'r') as f:
    encoder = model_from_json(f.read())
    encoder.load_weights("encoder.h5")
    
with open('autoencoder.json', 'r') as f:
    autoencoder = model_from_json(f.read())
    autoencoder.load_weights("autoencoder.h5")
    
with open('generator.json', 'r') as f:
    generator = model_from_json(f.read(), {'custom_activation': custom_activation})
    generator.load_weights("generator.h5")

with open('discriminator.json', 'r') as f:
    discriminator = model_from_json(f.read())
    discriminator.load_weights("discriminator.h5")
    
generator_per_class = []
discriminator_per_class = []
for _class in range(10):
    with open('models/generator%s.json' % str(_class), 'r') as f:
        generatora = model_from_json(f.read(), {'custom_activation': custom_activation})
        generatora.load_weights("models/generator%s.h5" % str(_class))
        generator_per_class.append(generatora)

    with open('models/discriminator%s.json' % str(_class), 'r') as f:
        discriminatora = model_from_json(f.read())
        discriminatora.load_weights("models/discriminator%s.h5" % str(_class))
        discriminator_per_class.append(discriminatora)

/home/ganimedes/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))

x_train = encoder.predict(x_train)

In [3]:
print(x_train.shape)
print(y_train.shape)

(60000, 4, 4, 4)
(60000,)


In [4]:
from sklearn import mixture

(x_train, y_train), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))

x_train = encoder.predict(x_train)

In [5]:
print(x_train.shape)

(60000, 4, 4, 4)


In [6]:
x_train = x_train.reshape(60000, 64)

In [7]:
from sklearn import mixture

(x_train, y_train), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))

x_train = encoder.predict(x_train)

data_gmm = x_train.reshape(x_train.shape[0], -1)

gmm_model = []
for i in range(10):
    gmm_model.append(mixture.GaussianMixture(n_components=10, covariance_type='full'))
    result = []
    for a, b in zip(data_gmm, y_train):
        if b == i:
            result.append(a)
    data_train = np.array(result)
    gmm_model[i].fit(data_train)

In [22]:
def gmm_generator(samples_count, class_type):
    a = gmm_model[class_type].sample(samples_count)
    sample = np.array(a[0])
    return sample


def cluster_per_feature_class_generator(samples_count, class_type):
    data = x_train.reshape(x_train.shape[0], -1)
    result = []
    for a, b in zip(data, y_train):
        if b == class_type:
            result.append(a)
    data = np.array(result).T
    result = []
    for a in data:
        result.append(np.random.choice(a, samples_count))
    result = np.array(result)
    result = result.T
    return result
            
def cluster_per_feature_generator(samples_count):
    data = x_train.reshape(x_train.shape[0], -1)
    data = data.T
    result = []
    for a in data:
        result.append(np.random.choice(a, samples_count))
    result = np.array(result)
    result = result.T
    return result

def generate_per_class(samples_count, class_type):
    return generator_per_class[class_type].predict(np.random.normal(0, 1, (samples_count, 100)))

def discriminator_vote(samples):
    #(batch, 64)
    temp = []
    for disc in discriminator_per_class:
        prediction = disc.predict(samples)
        prediction = np.round(prediction, 0)
        temp.append(prediction)
    temp = np.array(temp)
    temp = temp.T.reshape(-1, 10)
    print(temp.shape)
    result = []
    for row in temp:
        if np.sum(row) == 0:
            result.append(0)
        else:
            result.append(1)
    return np.array(result)
    

In [9]:
generator_count = 5

real_x = x_train.reshape(x_train.shape[0], -1)

def prepare_data_same_ratio_of_generators(dim):
    samples_per_generator_count = dim // generator_count
    data_x = np.concatenate([real_x[:samples_per_generator_count], generator.predict(np.random.normal(0, 1, (samples_per_generator_count, 100)))])
    for _class in range(10):
        data_x = np.concatenate([data_x, gmm_generator(samples_per_generator_count // 10, _class), cluster_per_feature_class_generator(samples_per_generator_count // 10, _class), generate_per_class(samples_per_generator_count // 10, _class)])
    data_y = np.concatenate([np.ones(samples_per_generator_count), np.zeros((generator_count - 1)*samples_per_generator_count)])
    s = np.arange(data_x.shape[0])
    np.random.shuffle(s)
    return data_x[s], data_y[s]

def prepare_data_same_count_of_fakes_to_real(dim):
    samples_per_generator_count = (dim // 2) // (generator_count - 1)
    data_x = np.concatenate([real_x[:dim // 2], generator.predict(np.random.normal(0, 1, (samples_per_generator_count, 100)))])
    for _class in range(10):
        data_x = np.concatenate([data_x, gmm_generator(samples_per_generator_count // 10, _class), cluster_per_feature_class_generator(samples_per_generator_count // 10, _class), generate_per_class(samples_per_generator_count // 10, _class)])
    data_y = np.concatenate([np.ones(dim // 2), np.zeros(dim // 2)])
    s = np.arange(data_x.shape[0])
    np.random.shuffle(s)
    return data_x[s], data_y[s]
    

In [ ]:
train_x, train_y = prepare_data_same_ratio_of_generators(50000)
validate_x, validate_y = prepare_data_same_ratio_of_generators(10000)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

clf = SVC()
clf.fit(train_x, train_y)
predictions = clf.predict(validate_x)
print(accuracy_score(validate_y, predictions))
print(confusion_matrix(validate_y, predictions))
print(classification_report(validate_y, predictions))

In [ ]:
predictions = discriminator_vote(validate_x)
print(accuracy_score(validate_y, predictions))
print(confusion_matrix(validate_y, predictions))
print(classification_report(validate_y, predictions))

In [10]:
train_x, train_y = prepare_data_same_count_of_fakes_to_real(80000)
validate_x, validate_y = prepare_data_same_count_of_fakes_to_real(10000)

In [11]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

clf = SVC(verbose=True)
clf.fit(train_x, train_y)
predictions = clf.predict(validate_x)
print(accuracy_score(validate_y, predictions))
print(confusion_matrix(validate_y, predictions))
print(classification_report(validate_y, predictions))

[LibSVM]0.7133
[[2554 2446]
 [ 421 4579]]
             precision    recall  f1-score   support

        0.0       0.86      0.51      0.64      5000
        1.0       0.65      0.92      0.76      5000

avg / total       0.76      0.71      0.70     10000



In [23]:
predictions = discriminator_vote(validate_x).reshape(-1)
print(predictions.shape)
print(accuracy_score(validate_y, predictions))
print(confusion_matrix(validate_y, predictions))
print(classification_report(validate_y, predictions))

(10000, 10)
(10000,)
0.5619
[[ 661 4339]
 [  42 4958]]
             precision    recall  f1-score   support

        0.0       0.94      0.13      0.23      5000
        1.0       0.53      0.99      0.69      5000

avg / total       0.74      0.56      0.46     10000



In [29]:
clf = discriminator
clf.compile(optimizer='adadelta', loss='binary_crossentropy')
clf.fit(train_x, train_y,
                epochs=30,
                batch_size=128,
                shuffle=True,
                validation_data=(validate_x, validate_y))

Train on 80000 samples, validate on 10000 samples
Epoch 1/30
80000/80000 [==============================] - 5s - loss: 0.5478 - val_loss: 0.5100
Epoch 2/30
80000/80000 [==============================] - 6s - loss: 0.5010 - val_loss: 0.4853
Epoch 3/30
80000/80000 [==============================] - 5s - loss: 0.4736 - val_loss: 0.4978
Epoch 4/30
80000/80000 [==============================] - 5s - loss: 0.4555 - val_loss: 0.4472
Epoch 5/30
80000/80000 [==============================] - 5s - loss: 0.4382 - val_loss: 0.4402
Epoch 6/30
80000/80000 [==============================] - 5s - loss: 0.4222 - val_loss: 0.4241
Epoch 7/30
80000/80000 [==============================] - 6s - loss: 0.4083 - val_loss: 0.4403
Epoch 8/30
80000/80000 [==============================] - 6s - loss: 0.3967 - val_loss: 0.3922
Epoch 9/30
80000/80000 [==============================] - 6s - loss: 0.3851 - val_loss: 0.3849
Epoch 10/30
80000/80000 [==============================] - 5s - loss: 0.3715 - val_loss: 0.3911

In [ ]:
predictions = clf.predict(validate_x)
predictions = np.round(predictions, 0)
print(accuracy_score(validate_y, predictions))
print(confusion_matrix(validate_y, predictions))
print(classification_report(validate_y, predictions))

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(train_x, train_y)
predictions = clf.predict(validate_x)
print(accuracy_score(validate_y, predictions))
print(confusion_matrix(validate_y, predictions))
print(classification_report(validate_y, predictions))

In [24]:
gen1 = generator.predict(np.random.normal(0, 1, (10000, 100)))
gen2 = cluster_per_feature_generator(10000)
gen3 = gmm_generator(1000, 9)
gen4 = cluster_per_feature_class_generator(1000, 9)
gen5 = generate_per_class(1000, 9)
for a in range(9):
    gen3 = np.concatenate([gen3, gmm_generator(1000, a)])
    gen4 = np.concatenate([gen4, cluster_per_feature_class_generator(1000, a)])
    gen5 = np.concatenate([gen3, generate_per_class(1000, a)])

In [34]:
result = clf.predict(gen2)
result = np.round(result, 0)
count = 0
for a in result:
    if a == 0:
        count = count + 1
print(count / result.size)

0.9994


In [35]:
result = clf.predict(gen3)
result = np.round(result, 0)

count = 0
for a in result:
    if a == 0:
        count = count + 1
print(count / result.size)

0.526


In [36]:
result = clf.predict(gen1)
result = np.round(result, 0)

count = 0
for a in result:
    if a == 0:
        count = count + 1
print(count / result.size)

0.8811


In [37]:
result = clf.predict(gen5)
result = np.round(result, 0)

count = 0
for a in result:
    if a == 0:
        count = count + 1
print(count / result.size)

0.5309090909090909
